In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --build-file test.spi test.fsx --timeout 10000 } | Invoke-Block

00:00:00 #1 [Debug] runWithTimeoutAsync / timeout: 60
00:00:00 #2 [Debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@411-297> }
00:00:00 #3 [Verbose] > pwd: C:\home\git\polyglot\apps\spiral\temp\test
00:00:00 #4 [Verbose] > dll_path: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:01 #5 [Verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:01 #6 [Debug] runWithTimeoutAsync / timeout: 500
00:00:01 #7 [Verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:01 #8 [Debug] sendJson / port: 13805 / json: {"Ping":true} / result.Length:
00:00:01 #9 [Verbose] awaitCompiler / Ping / result: Some(null) / 

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.13.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @ncave
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
.> cmd /C dotnet restore test.fable-temp.csproj -p:FABLE_COMPILER=true -p:FABLE_COMPILER_4=true -p:FABLE_COMPILER_RUST=true
  Determining projects to restore...
  Restored C:\home\git\polyglot\apps\spiral\temp\test\test.fable-temp.csproj (in 352 ms).
.> cmd /C dotnet restore C:/home/git/polyglot/apps/spiral/temp/test/test.fsproj
  Determining projects to restore...
  Restored C:\home\git\polyglot\apps\spiral\temp\test\test.fsproj (in 369 ms).
Project and references (1 source files) parsed in 8492ms

Started Fable compilation...
Fable compilation finished in 2134ms

.\test.fsx(6,0): (6,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
| Set-Content test.rs

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

   Compiling test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
   --> apps\spiral\temp\test\./main.rs:190:15
    |
190 | use tracing::{info, Level};
    |               ^^^^
    |
    = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:46:8
   |
45 | impl Cart {
   | --------- associated items in this implementation
46 |     fn new() -> Cart {
   |        ^^^
...
50 |     fn add_item(&mut self, item: Item) {
   |        ^^^^^^^^
...
56 |     fn remove_item(&mut self, item: &Item) {
   |        ^^^^^^^^^^^
   |
   = note: `#[warn(dead_code)]` on by default

   --> apps\spiral\temp\test\./main.rs:124:4
    |
124 | fn parse_comment(input: &str) -> IResult<&str, SpiralToken> {
    |    ^^^^^^^^^^^^^

   --> apps\spiral\temp\test\./main.rs:130:4
    |
130 | fn parse_string(input: &str) -> IResult<&str, SpiralToken> {
    |    ^^^^^^^^^^^^

   --> apps\spiral\temp\test\./main.rs:145:4
    |
145 | fn parse_identifier(input: &str) -> IResult<&str, Spir

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling fable_library_rust v0.1.0 (C:\home\git\polyglot\apps\spiral\temp\test\fable_modules\fable-library-rust)
   Compiling test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
    Finished release [optimized] target(s) in 51.05s
     Running unittests main.rs (C:\home\git\polyglot\target\release\deps\test-cc99bcdf5fa61b70.exe)

running 3 tests
test test_parse_number ... ok
2024-02-23T22:04:17.305027Z  INFO test: input=Operator("=")
2024-02-23T22:04:17.305174Z  INFO test: input=Operator("=")
2024-02-23T22:04:17.305332Z  INFO test: input=Operator(")")
2024-02-23T22:04:17.305533Z  INFO test: input=Integer(-5136517121875230874)
2024-02-23T22:04:17.305709Z  INFO test: input=StringLiteral("o4@S@:KK~g|d0/z.3R6")
2024-02-23T22:04:17.305859Z  INFO test: input=StringLiteral("=|3:(E$ELQQ`]~?k>{zsD{")
2024-02-23T22:04:17.305976Z  INFO test: input=Comment("<")
2024-02-23T22:04:17.306045Z  INFO test: input=Operator("/")
2024-02-23T22:04:17.306123Z  INFO test: input=StringLiteral("&%~Er@UX

In [ ]:
{ . $ScriptDir/../../../../target/release/test$(GetExecutableSuffix) } | Invoke-Block

app=test
